Afinamiento de parámetros con Keras Tuner --- 0:00 min
===

* Última modificación: Marzo 7, 2022 | YouTube

Importación de librerías
---

In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf

Preparación del dataset
--

In [4]:
(img_train, label_train), (
    img_test,
    label_test,
) = tf.keras.datasets.fashion_mnist.load_data()

In [5]:
img_train = img_train.astype("float32") / 255.0
img_test = img_test.astype("float32") / 255.0

Definición del modelo
---

In [12]:
def model_builder(hp):

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

    hp_units = hp.Int(
        "units",
        min_value=32,
        max_value=512,
        step=32,
    )

    model.add(tf.keras.layers.Dense(units=hp_units, activation="relu"))
    model.add(tf.keras.layers.Dense(10))

    hp_learning_rate = hp.Choice(
        "learning_rate",
        values=[
            1e-2,
            1e-3,
            1e-4,
        ],
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp_learning_rate,
        ),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True,
        ),
        metrics=["accuracy"],
    )

    return model

Uso
--

In [13]:
import keras_tuner as kt

tuner = kt.Hyperband(
    model_builder,
    objective="val_accuracy",
    max_epochs=10,
    factor=3,
    directory="/tmp/my_dir",
    project_name="intro_to_kt",
)

In [14]:
stop_early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
)

In [15]:
tuner.search(
    img_train,
    label_train,
    epochs=50,
    validation_split=0.2,
    callbacks=[
        stop_early,
    ],
)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(
    f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
"""
)

Trial 30 Complete [00h 00m 51s]
val_accuracy: 0.8809166550636292

Best val_accuracy So Far: 0.8892499804496765
Total elapsed time: 00h 09m 01s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 320 and the optimal learning rate for the optimizer
is 0.001.



Entrenamiento del modelo
---

In [16]:
model = tuner.hypermodel.build(best_hps)

history = model.fit(
    img_train,
    label_train,
    epochs=50,
    validation_split=0.2,
)

val_acc_per_epoch = history.history["val_accuracy"]
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print("Best epoch: %d" % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 6s 3ms/step - loss: 0.4979 - accuracy: 0.8216 - val_loss: 0.4029 - val_accuracy: 0.8551
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3743 - accuracy: 0.8642 - val_loss: 0.3566 - val_accuracy: 0.8702
Epoch 3/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3334 - accuracy: 0.8772 - val_loss: 0.3902 - val_accuracy: 0.8612
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3082 - accuracy: 0.8868 - val_loss: 0.3343 - val_accuracy: 0.8817
Epoch 5/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2867 - accuracy: 0.8939 - val_loss: 0.3279 - val_accuracy: 0.8799
Epoch 6/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2730 - accuracy: 0.8966 - val_loss: 0.3187 - val_accuracy: 0.8862
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2602 - accuracy: 0.9019 - val_loss: 0.3247 - val_accuracy:

In [17]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(
    img_train,
    label_train,
    epochs=best_epoch,
    validation_split=0.2,
)

Epoch 1/48
1500/1500 [==============================] - 6s 3ms/step - loss: 0.5022 - accuracy: 0.8236 - val_loss: 0.4057 - val_accuracy: 0.8521
Epoch 2/48
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3715 - accuracy: 0.8640 - val_loss: 0.3625 - val_accuracy: 0.8698
Epoch 3/48
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3353 - accuracy: 0.8764 - val_loss: 0.3375 - val_accuracy: 0.8752
Epoch 4/48
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3066 - accuracy: 0.8873 - val_loss: 0.3449 - val_accuracy: 0.8766
Epoch 5/48
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2885 - accuracy: 0.8919 - val_loss: 0.3469 - val_accuracy: 0.8770
Epoch 6/48
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2720 - accuracy: 0.8980 - val_loss: 0.3300 - val_accuracy: 0.8823
Epoch 7/48
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2621 - accuracy: 0.9020 - val_loss: 0.3190 - val_accuracy:

In [18]:
eval_result = hypermodel.evaluate(
    img_test,
    label_test,
)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.5936 - accuracy: 0.8819
[test loss, test accuracy]: [0.5936309099197388, 0.8819000124931335]
